# 문장 토큰 단위 분류 모델 학습

## 1. CPU 및 GPU 환경설정

In [ ]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.cuda.get_device_name(0)

## 2. 데이터셋

* 한국해양대에서 공개한 개체명 인식 데이터셋

In [ ]:
!git clone https://github.com/kmounlp/NER.git

In [ ]:
import os
import glob

In [ ]:
file_list = []

In [ ]:
for x in os.walk('/content/NER/'):
    for y in glob.glob(os.path.join(x[0], '*_NER.txt')):    # ner.*, *_NER.txt
        file_list.append(y)

In [ ]:
file_list = sorted(file_list)

* 데이터셋 확인
  * 개체명인식 데이터셋이 여러 파일로 분류되어 있음

In [ ]:
for file_path in file_list:
    print(file_path)

## 3. 허깅페이스 트랜스포머 설치

In [ ]:
!pip install transformers

## 4. 데이터셋 샘플

In [ ]:
from pathlib import Path

In [ ]:
file_path = file_list[0]
file_path = Path(file_path)
raw_text = file_path.read_text().strip()

* 데이터셋 샘플 확인
  * 형태소 단위로 tokenizing되어 있음
  * 'B', 'I', 'O' 형태로 태그가 부착되어 있음

In [ ]:
print(raw_text[0:1000])
'''
## 1
## 오에 겐자부로는 일본 현대문학의 초석을 놓은 것으로 평가받는 작가 나쓰메 소세키(1867~1916)의 대표작 ‘마음’에 담긴 군국주의적 요소, 야스쿠니 신사 참배 행위까지 소설의 삽화로 동원하며 일본 사회의 ‘비정상성’을 문제 삼는다.
## <오에 겐자부로:PER>는 <일본:LOC> 현대문학의 초석을 놓은 것으로 평가받는 작가 <나쓰메 소세키:PER>(<1867~1916:DUR>)의 대표작 ‘<마음:POH>’에 담긴 군국주의적 요소, <야스쿠니 신사:ORG> 참배 행위까지 소설의 삽화로 동원하며 <일본:ORG> 사회의 ‘비정상성’을 문제 삼는다.
오에	오에	NNG	B-PER
_	_	_	I-PER
겐자부로	겐자부로	NNP	I-PER
는	는	JX	O
_	_	_	O
일본	일본	NNP	B-LOC
_	_	_	O
현대	현대	NNG	O
문학	문학	NNG	O
의	의	JKG	O
_	_	_	O
'''

## 5. 데이터셋 전처리

* 음절단위로 변환

* 현재 데이터셋의 형태는 형태소 단위로 enter로 구분되어 있음
* BERT로 입력하기 위해서 하나의 문장으로 합쳐야함
* 태그는 각각 token에 맞는 label로 넣어줘야함

* 전처리 과정
  * 데이터셋을 전부 읽고 이중엔터를 기준으로 document로 구분함
  * 각 line을 읽으면서 token들을 문장으로 부착함

In [ ]:
import re

In [ ]:
def read_file(file_list):
    token_docs = []
    tag_docs = []
    for file_path in file_list:
        # print("read file from ", file_path)
        file_path = Path(file_path)
        raw_text = file_path.read_text().strip()
        raw_docs = re.split(r'\n\t?\n', raw_text)
        for doc in raw_docs:
            tokens = []
            tags = []
            for line in doc.split('\n'):
                if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
                    continue
                try:
                    token = line.split('\t')[0]
                    tag = line.split('\t')[3]   # 2: pos, 3: ner
                    for i, syllable in enumerate(token):    # 음절 단위로 token을 자름
                        tokens.append(syllable) # 음절 단위 정보를 가져와서 문장을 만듬
                        modi_tag = tag
                        if i > 0:
                            if tag[0] == 'B':
                                modi_tag = 'I' + tag[1:]    # 음절에 대해 BIO tag를 부착함
                        tags.append(modi_tag)
                except:
                    print(line)
            token_docs.append(tokens)
            tag_docs.append(tags)

    return token_docs, tag_docs

In [ ]:
texts, tags = read_file(file_list[:])

In [ ]:
print(len(texts)) # 19263
print(len(tags)) # 19263

* 데이터셋 확인

In [ ]:
print(texts[0], end='\n\n') # 음절 단위로 잘 잘렸네요!
# ['오', '에', '_', '겐', '자', '부', '로', '는', '_', '일', '본', '_', '현', '대', '문', '학', '의', '_', '초', '석', '을', '_', '놓', '은', '_', '것', '으', '로', '_', '평', '가', '받', '는', '_', '작', '가', '_', '나', '쓰', '메', '_', '소', '세', '키', '(', '1', '8', '6', '7', '~', '1', '9', '1', '6', ')', '의', '_', '대', '표', '작', '_', '‘', '마', '음', '’', '에', '_', '담', '긴', '_', '군', '국', '주', '의', '적', '_', '요', '소', ',', '_', '야', '스', '쿠', '니', '_', '신', '사', '_', '참', '배', '_', '행', '위', '까', '지', '_', '소', '설', '의', '_', '삽', '화', '로', '_', '동', '원', '하', '며', '_', '일', '본', '_', '사', '회', '의', '_', '‘', '비', '정', '상', '성', '’', '을', '_', '문', '제', '_', '삼', '는', '다', '.']
print(tags[0])
# ['B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'B-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'I-DUR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

* 학습을 위해 tag를 (vocab id)처럼 tag label id로 바꿔줘야함

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

* 데이터가 제공하는 개체명 태그 종류 확인

In [ ]:
for i, tag in enumerate(unique_tags):
    print(tag)  # 학습을 위한 label list를 확인합니다.

'''
I-NOH
I-MNY
I-LOC
B-TIM
I-PNT
I-DAT
B-DAT
B-PER
I-POH
I-DUR
B-ORG
B-LOC
B-MNY
O
I-PER
I-ORG
B-POH
I-TIM
B-NOH
B-DUR
B-PNT
'''

## 6. EDA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
texts_len = [len(x) for x in texts]

### 1. 문장의 길이의 히스토그램

In [ ]:
plt.figure(figsize=(16,10))
plt.hist(texts_len, bins=50, range=[0,800], facecolor='b', density=True, label='Text Length')
plt.title('Text Length Histogram')
plt.legend()
plt.xlabel('Number of Words')
plt.ylabel('Probability')

### 2. 각 NER 태그별 데이터에 포함된 갯수

In [ ]:
for tag in list(tag2id.keys()) : 
    globals()[tag] = 0

In [ ]:
for tag in tags : 
    for ner in tag : 
        globals()[ner] += 1

* 개수가 부족한 태그에 대해서는 학습성능이 떨어짐
  * 해당 태그에 관련된 데이터셋을 더 추가하여 보완함

In [ ]:
for tag in list(tag2id.keys()) : 
    print('{:>6} : {:>7,}'. format(tag, globals()[tag]))
'''
 I-NOH :  23,967
 I-MNY :   6,930
 I-LOC :  16,537
 B-TIM :     371
 I-PNT :   4,613
 I-DAT :  14,433
 B-DAT :   5,383
 B-PER :  13,779
 I-POH :  37,156
 I-DUR :   4,573
 B-ORG :  13,089
 B-LOC :   6,313
 B-MNY :   1,440
     O : 983,746
 I-PER :  26,206
 I-ORG :  41,320
 B-POH :   6,686
 I-TIM :   1,876
 B-NOH :  11,051
 B-DUR :   1,207
 B-PNT :   1,672
'''

## 7. Train Test Split

* train과 test 데이터셋을 나누고 학습을 위한 데이터셋 만듬
  * train : 80%, test : 20%

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_tags, test_tags = train_test_split(texts, tags, test_size=.2) 

In [ ]:
print('Train 문장 : {:>6,}' .format(len(train_texts)))
# Train 문장 : 15,410
print('Train 태그 : {:>6,}' .format(len(train_tags)))
# Train 태그 : 15,410
print('Test  문장 : {:>6,}' .format(len(test_texts)))
# Test  문장 :  3,853
print('Test  태그 : {:>6,}' .format(len(test_tags)))
# Test  태그 :  3,853

## 8. BERT 토크나이저

In [ ]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

* [CLS], [PAD], [SEP] token의 label이 데이터셋에 존재하지 않기 때문에 임의로 데이터셋을 만들어줌
  * 'O' 태그로 label 지정

In [ ]:
pad_token_id = tokenizer.pad_token_id # 0
cls_token_id = tokenizer.cls_token_id # 101
sep_token_id = tokenizer.sep_token_id # 102
pad_token_label_id = tag2id['O']    # tag2id['O']
cls_token_label_id = tag2id['O']
sep_token_label_id = tag2id['O']

* 현재는 음절단위로 나눴기 때문에 wordPiece tokenizer를 사용하면 다른 결과가 나옴
* 음절 단위 tokenizer를 만들어서 사용

* 음절단위 tokenizer 함수 구현
  * 중간 음절에는 모두 prefix(##)을 붙임
  * 기존 tokenizer와 동일한 return값을 가짐

* 'bert-base-multilingual-cased' 모델은 한국어가 대부분 음절단위이기 때문에 음절단위 tokenizer를 적용해도 vocab id를 대부분 획득할 수 있음

In [ ]:
# 기존 토크나이저는 wordPiece tokenizer로 tokenizing 결과를 반환합니다.
# 데이터 단위를 음절 단위로 변경했기 때문에, tokenizer도 음절 tokenizer로 바꿀게요! :-)

def ner_tokenizer(sent, max_seq_length):    
    pre_syllable = "_"
    input_ids = [pad_token_id] * (max_seq_length - 1)
    attention_mask = [0] * (max_seq_length - 1)
    token_type_ids = [0] * max_seq_length
    sent = sent[:max_seq_length-2]

    for i, syllable in enumerate(sent):
        if syllable == '_':
            pre_syllable = syllable
        if pre_syllable != "_":
            syllable = '##' + syllable  # 중간 음절에는 모두 prefix를 붙입니다.
            # 이순신은 조선 -> [이, ##순, ##신, ##은, 조, ##선]
        pre_syllable = syllable

        input_ids[i] = (tokenizer.convert_tokens_to_ids(syllable))
        attention_mask[i] = 1
    
    input_ids = [cls_token_id] + input_ids
    input_ids[len(sent)+1] = sep_token_id
    attention_mask = [1] + attention_mask
    attention_mask[len(sent)+1] = 1
    return {"input_ids":input_ids,
            "attention_mask":attention_mask,
            "token_type_ids":token_type_ids}

In [ ]:
print(ner_tokenizer(train_texts[0], 5))
# {'input_ids': [101, 9954, 20479, 37824, 102], 'attention_mask': [1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0]}

In [ ]:
tokenized_train_sentences = []
tokenized_test_sentences = []
for text in train_texts:    # 전체 데이터를 tokenizing 합니다.
    tokenized_train_sentences.append(ner_tokenizer(text, 128))
for text in test_texts:
    tokenized_test_sentences.append(ner_tokenizer(text, 128))


* `encode_tags()`
  * label을 truncation과 padding과정이 포함된 함수
  * tokenizer에 truncation과 padding과정이 포함되어 있기 때문에, label 데이터도 truncation과 padding과정이 필요함

In [ ]:
def encode_tags(tags, max_seq_length):
    # label 역시 입력 token과 개수를 맞춰줍니다 :-)
    tags = tags[:max_seq_length-2]
    labels = [tag2id[tag] for tag in tags]
    labels = [tag2id['O']] + labels

    padding_length = max_seq_length - len(labels)
    labels = labels + ([pad_token_label_id] * padding_length)

    return labels

In [ ]:
encode_tags(train_tags[0], 5)
# [13, 10, 15, 15, 13]

In [ ]:
train_labels = []
test_labels = []

for tag in train_tags:
    train_labels.append(encode_tags(tag, 128))

for tag in test_tags:
    test_labels.append(encode_tags(tag, 128))


In [ ]:
len(train_labels), len(test_labels)
# (15410, 3853)

## 9. Token 데이터셋

* TokenDataset 구현
  * `__getitem__()`
    * input이 들어옴
    * 사전에 정의된 label이 순차적으로 들어감

In [ ]:
import torch

class TokenDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val) for key, val in self.encodings[idx].items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TokenDataset(tokenized_train_sentences, train_labels)
test_dataset = TokenDataset(tokenized_test_sentences, test_labels)

In [ ]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments
import sys
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    learning_rate=3e-5,
    save_total_limit=5
)

## 10. BertForTokenClassification

* 각각의 token 마다 classification이 부착되어 해당 token이 어떤 label 값인지 분류하는 과정을 진행함

* model이 TokenDataset을 가져와서 학습을 진행함

* model initialize
  * `num_labels` : 구분해야하는 label의 개수 지정

In [ ]:
model = BertForTokenClassification.from_pretrained(MODEL_NAME, num_labels=len(unique_tags))
model.to(device)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

In [ ]:
trainer.train() # 1 epoch에 대략 5분 정도 걸립니다.

## 11. New Data Inference

* 음절 tokenizer를 사용하여 학습했기 때문에
* inference에서도 입력된 문장에 대해서 음절 tokenizer를 거친 후에 model의 입력으로 들어가야함

In [ ]:
def ner_inference(text) : 
  
    model.eval()
    text = text.replace(' ', '_')

    predictions , true_labels = [], []
    
    tokenized_sent = ner_tokenizer(text, len(text)+2)
    input_ids = torch.tensor(tokenized_sent['input_ids']).unsqueeze(0).to(device)
    attention_mask = torch.tensor(tokenized_sent['attention_mask']).unsqueeze(0).to(device)
    token_type_ids = torch.tensor(tokenized_sent['token_type_ids']).unsqueeze(0).to(device)    
    
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        
    logits = outputs['logits']
    logits = logits.detach().cpu().numpy()
    label_ids = token_type_ids.cpu().numpy()

    predictions.extend([list(p) for p in np.argmax(logits, axis=2)]) # 각 token에 대해 softmax가 최대로 되는 값이 무엇인지 가져와서 token 결과 return
    true_labels.append(label_ids)

    pred_tags = [list(tag2id.keys())[p_i] for p in predictions for p_i in p]

    print('{}\t{}'.format("TOKEN", "TAG"))
    print("===========")
    # for token, tag in zip(tokenizer.decode(tokenized_sent['input_ids']), pred_tags):
    #   print("{:^5}\t{:^5}".format(token, tag))
    for i, tag in enumerate(pred_tags):
        print("{:^5}\t{:^5}".format(tokenizer.convert_ids_to_tokens(tokenized_sent['input_ids'][i]), tag))

In [ ]:
text = '이순신은 조선 중기의 무신이다.'

In [ ]:
ner_inference(text)
'''
TOKEN	TAG
===========
[CLS]	  O  
  이  	B-PER
 ##순 	I-PER
 ##신 	I-PER
 ##은 	  O  
  _  	  O  
  조  	  O  
 ##선 	  O  
  _  	  O  
  중  	  O  
 ##기 	  O  
 ##의 	  O  
  _  	  O  
  무  	  O  
 ##신 	  O  
 ##이 	  O  
 ##다 	  O  
 ##. 	  O  
[SEP]	  O  
'''

In [ ]:
text = '로스트아크는 스마일게이트 RPG가 개발한 쿼터뷰 액션 MMORPG 게임이다.'
ner_inference(text)
'''
TOKEN	TAG
===========
[CLS]	  O  
  로  	B-POH
 ##스 	I-POH
 ##트 	I-POH
 ##아 	I-POH
 ##크 	I-POH
 ##는 	  O  
  _  	  O  
  스  	B-ORG
 ##마 	I-ORG
 ##일 	I-ORG
 ##게 	I-ORG
 ##이 	I-ORG
 ##트 	I-ORG
  _  	  O  
  R  	  O  
 ##P 	  O  
 ##G 	  O  
 ##가 	  O  
  _  	  O  
  개  	  O  
 ##발 	  O  
 ##한 	  O  
  _  	  O  
  쿼  	  O  
 ##터 	  O  
 ##뷰 	  O  
  _  	  O  
  액  	  O  
 ##션 	  O  
  _  	  O  
  M  	  O  
 ##M 	  O  
 ##O 	  O  
 ##R 	  O  
 ##P 	  O  
 ##G 	  O  
  _  	  O  
  게  	  O  
 ##임 	  O  
 ##이 	  O  
 ##다 	  O  
 ##. 	  O  
[SEP]	  O  
'''

In [ ]:
text = '2014년 11월 12일 최초 공개했으며 2018년 11월 7일부터 오픈 베타 테스트를 진행하다 2019년 12월 4일 정식 오픈했다.'
ner_inference(text)

In [ ]:
text = '짜장면 7,000원'
ner_inference(text)
'''
TOKEN	TAG
===========
[CLS]	  O  
  짜  	  O  
 ##장 	  O  
 ##면 	  O  
  _  	  O  
  7  	B-MNY
 ##, 	  O  
 ##0 	I-MNY
 ##0 	I-MNY
 ##0 	I-MNY
 ##원 	I-MNY
[SEP]	  O  
'''